In [2]:
from pathlib import Path
import sys
import os.path
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import f_oneway, gaussian_kde, mannwhitneyu, pearsonr, spearmanr
import seaborn as sns
from tqdm.notebook import tqdm
from pymodulon.compare import compare_ica
from pymodulon.core import IcaData
from pymodulon.io import load_json_model, save_to_json
from pymodulon.plotting import *

import json

from pymodulon.util import explained_variance
from pymodulon.imodulondb import imdb_gene_presence

%matplotlib inline
%config  InlineBackend.figure_format = 'png'

sns.set_style('whitegrid')

In [53]:
Fast_ICA_Gene_Presence_Matrix = pd.read_csv("./Presence_Matrices/Fast_ICA_Gene_Presence_Matrix.csv", index_col=0).astype(int)

In [25]:
Hierarchical_Gene_Presence_Matrix = pd.read_csv("./Presence_Matrices/Corr_Hierarchical_Presence_Matrix.csv", index_col=0).astype(int)

In [26]:
Biclustering_Presence_Matrix = pd.read_csv("./Presence_Matrices/Biclustering_Presence_Matrix.csv", index_col=0).astype(int)

In [43]:
WGCNA_Gene_Presence_Matrix = pd.read_csv("./Presence_Matrices/WGCNA_Gene_Presence_Matrix.csv", index_col=0).astype(int)

In [59]:
FLAME_Gene_Presence_Matrix = pd.read_csv("./Presence_Matrices/FLAME_Gene_Presence_Matrix.csv", index_col=0).astype(int)

In [58]:
QUBIC_Gene_Presence_Matrix = pd.read_csv("./Presence_Matrices/QUBIC_Gene_Presence_Matrix.csv", index_col=0).astype(int)

In [42]:
Hierarchical_Gene_Presence_Matrix

,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7,Cluster_8,Cluster_9,Cluster_10,...,Cluster_241,Cluster_242,Cluster_243,Cluster_244,Cluster_245,Cluster_246,Cluster_247,Cluster_248,Cluster_249,Cluster_250
b0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b4747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b4748,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b4751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b4755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
Biclustering_Presence_Matrix

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
b0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b4747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b4748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b4751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b4755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
precise1k = load_json_model('../Data/p1k.json.gz')

In [27]:
regulon_p1k = pd.crosstab(precise1k.trn['gene_id'], precise1k.trn['regulator'])

In [28]:
regulon_p1k

regulator,AccB,AccDA,AcnB,AcrR,Ada,Adenosylcobalamin,AdiY,AgaR,AidB,AlaS,...,rydC,ryfA,ryfD,ryhB,sdsR,sgrS,sokB,sokC,spf,symR
gene_id,,,,,,,,,,,,,,,,,,,,,
b0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b0010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b4705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b4720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b4721,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Hierarchical

In [39]:
import pandas as pd
import numpy as np

def calculate_metrics(y_true, y_pred):
    # True positives, false positives, true negatives, false negatives
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    # Precision, Recall, F1-score calculations
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # MCC calculation with proper handling of zero division
    denominator = np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    mcc = ((TP * TN) - (FP * FN)) / denominator if denominator > 0 else 0

    return precision, recall, f1, mcc

def find_best_match_and_evaluate(regulon_df, clusters_df):
    # Find the intersection of genes (rows) between the two dataframes
    common_genes = regulon_df.index.intersection(clusters_df.index)

    # Subset both dataframes to include only common genes
    regulon_df = regulon_df.loc[common_genes]
    clusters_df = clusters_df.loc[common_genes]

    # Store overall metrics to calculate averages later
    total_precision, total_recall, total_f1, total_mcc = 0, 0, 0, 0
    num_clusters = clusters_df.shape[1]  # Number of clusters

    # Iterate over clusters
    for cluster in clusters_df.columns:
        cluster_genes = clusters_df[cluster]

        best_f1 = -1  # Initialize best F1 to a low value
        best_metrics = None  # Placeholder for the best metrics for this cluster

        # Compare with each regulator in regulon_p1k
        for regulator in regulon_df.columns:
            regulator_genes = regulon_df[regulator]

            # Calculate metrics manually
            precision, recall, f1, mcc = calculate_metrics(cluster_genes.values, regulator_genes.values)

            # If this regulator gives the best F1-score so far, store its metrics
            if f1 > best_f1:
                best_f1 = f1
                best_metrics = (precision, recall, f1, mcc)

        # Accumulate the best metrics for this cluster
        if best_metrics:
            total_precision += best_metrics[0]
            total_recall += best_metrics[1]
            total_f1 += best_metrics[2]
            total_mcc += best_metrics[3]

    # Calculate average metrics across all clusters
    avg_precision = total_precision / num_clusters
    avg_recall = total_recall / num_clusters
    avg_f1 = total_f1 / num_clusters
    avg_mcc = total_mcc / num_clusters

    # Return the average metrics as a dictionary
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1,
        'average_mcc': avg_mcc
    }

# Example usage (with appropriate DataFrame loading):
# avg_metrics = find_best_match_and_evaluate(regulon_p1k, Hierarchical_Gene_Presence_Matrix)
# print(avg_metrics)


In [40]:
find_best_match_and_evaluate(regulon_p1k, Hierarchical_Gene_Presence_Matrix)

{'average_precision': 0.38968531443421994,
 'average_recall': 0.3639223239035935,
 'average_f1_score': 0.25884341568435065,
 'average_mcc': 0.2965072021648818}

In [41]:
find_best_match_and_evaluate(regulon_p1k, Biclustering_Presence_Matrix)

{'average_precision': 0.3234289527078748,
 'average_recall': 0.28501500809434055,
 'average_f1_score': 0.20104188088428712,
 'average_mcc': 0.23638978932577695}

In [44]:
find_best_match_and_evaluate(regulon_p1k, WGCNA_Gene_Presence_Matrix)

{'average_precision': 0.3250719790934106,
 'average_recall': 0.5746600556846481,
 'average_f1_score': 0.3076295905483286,
 'average_mcc': 0.35112262597032207}

In [61]:
find_best_match_and_evaluate(regulon_p1k, FLAME_Gene_Presence_Matrix)

{'average_precision': 0.3419231442988966,
 'average_recall': 0.5729531443645874,
 'average_f1_score': 0.3098195431588431,
 'average_mcc': 0.35406187989669113}

In [60]:
find_best_match_and_evaluate(regulon_p1k, QUBIC_Gene_Presence_Matrix)

{'average_precision': 0.4426851514071256,
 'average_recall': 0.27806750785407114,
 'average_f1_score': 0.2567641603095967,
 'average_mcc': 0.2868676595513832}